In [1]:
import os
import json

from prompt_formatter import create_prompt_from_file
from query_llm import query_llm
from utils import execute_query, TripletClause

import relationalai as rai
from relationalai.std import alias

In [2]:
!rai version


---------------------------------------------------
 
▰▱▱▱ Checking app version[▰▰▱▱ Checking app version[▰▰▰▱ Checking app version[▰▰▰▰ Checking app version[▱▰▰▰ Checking app version[▱▱▰▰ Checking app version[▱▱▱▰ Checking app version[▱▱▱▱ Checking app version[▰▱▱▱ Checking app version[▰▰▱▱ Checking app version[▰▰▰▱ Checking app version[▰▰▰▰ Checking app version[▱▰▰▰ Checking app version[▱▱▰▰ Checking app version[▱▱▱▰ Checking app version[▱▱▱▱ Checking app version[▰▱▱▱ Checking app version[▰▰▱▱ Checking app version[▰▰▰▱ Checking app version[▰▰▰▰ Checking app version[▱▰▰▰ Checking app version                                                                             
  RelationalAI   0.3.11              
  Rai-sdk        0.7.5               
  Python         3.11.0              
  App            2024.7.29-89bd1051  
                                     

---------------------------------------------------
 


In [3]:
root_dir=os.getcwd()

examples = {}
questions = {}
base_prompt_paths = {}


for head in ["ir", "pyrel"]:
    
    base_prompt_paths[head] = os.path.join(root_dir, f"heads/generate_{head}/base_prompt.txt")

    # Read examples from JSON file
    examples_path = os.path.join(root_dir, f"heads/generate_{head}/examples.json")
    with open(examples_path, 'r', encoding='utf-8') as file:
        examples[head] = json.load(file)

    # Read questions from JSON file
    questions_path = os.path.join(root_dir, f"heads/generate_{head}/questions.json")
    with open(questions_path, 'r', encoding='utf-8') as file:
        questions[head] = json.load(file)

## IR generation from NL query

In [4]:
head = "ir"
irs = []

for question in questions[head]:
    prompt = create_prompt_from_file(prompt_file_path=base_prompt_paths[head],
                                     examples=examples[head],
                                     question=question)
    ir = query_llm(task=head, 
                   prompt=prompt,
                   service_run=False)
    irs.append(ir)

print("\n\n".join(irs))

m: director(m, x); cast_member(m, y); spouse(x, y)

m: director(m, x); birth_place(x, y); cast_member(m, z); birth_place(z, y)

m: producer(m, x); cast_member(m, y); sibling(x, y)

m: director(m, x); cast_member(m, y); parent(x, y)

x, y: director(x, "Christopher Nolan"); director(y, "Steven Spielberg")

m1, m2: director(m1, "Steven Spielberg"); producer(m2, "Steven Spielberg")

m: director(m, "Quentin Tarantino"); cast_member(m, "Samuel L. Jackson"); cast_member(m, "Robert De Niro")
m: director(m, "Martin Scorsese"); cast_member(m, "Samuel L. Jackson"); cast_member(m, "Robert De Niro")

COUNT(m): director(m, "Christopher Nolan"); cast_member(m, "Christian Bale")


## PyRel generation from IR

In [5]:
head = "pyrel"

pyrel_queries = []

for question in questions[head]:
    prompt = create_prompt_from_file(prompt_file_path=base_prompt_paths[head],
                                     examples=examples[head],
                                     question=question)
    pyrel_query = query_llm(task=head, 
                            prompt=prompt,
                            service_run=False)
    
    pyrel_queries.append(pyrel_query)

print("\n\n".join(pyrel_queries))

with graph.query() as select:
    clause0 = clause(object_candidate_ids=None, relation_candidate_ids=["P57","P344"])
    clause1 = clause(object_candidate_ids=None, relation_candidate_ids=["P161","P674"])
    clause2 = clause(relation_candidate_ids=["P26","P8810"])
    clause0.source_ent_id == clause1.source_ent_id
    clause0.target_ent_id == clause2.source_ent_id
    clause1.target_ent_id == clause2.target_ent_id
    res_relations = select(alias(clause0.source_ent_id, "m"))
results = set(res_relations.results.get("m", []))
print(results)

with graph.query() as select:
    clause0 = clause(object_candidate_ids=None, relation_candidate_ids=["P57","P344"])
    clause1 = clause(object_candidate_ids=None, relation_candidate_ids=["P161","P674"])
    clause2 = clause(object_candidate_ids=None, relation_candidate_ids=["P19","P276"])
    clause3 = clause(object_candidate_ids=None, relation_candidate_ids=["P19","P276"])
    clause0.source_ent_id == clause1.source_ent_id
    clause0.target_ent_

## Build the RAI graph model and query it using PyRel


In [6]:
model_name = "QirKWikidata2907"
source = "faiss_db.data_schema"

* Ensure that you already created stream of data from SF to RAI. You need to do it once
```sh
    rai imports:stream --source faiss_db.data_schema.id_labels --model model_name
    rai imports:stream --source faiss_db.data_schema.triplets --model model_name
```

In [7]:
graph = rai.Model(model_name, isolated=False)
Labels = graph.Type("Labels", source=f"{source}.id_labels")
Facts = graph.Type("Facts", source=f"{source}.triplets")

In [8]:
Triplet = graph.Type("Triplet")

with graph.rule():
    f = Facts()
    Triplet.add(source_ent_id=f.source_ent_id, rid=f.rid, target_ent_id=f.target_ent_id)

In [9]:
clause = TripletClause(Triplet)
context = {"graph": graph, "clause": clause, "alias": alias}

In [10]:
responses_entity_id_format = []

for pyrel_query in pyrel_queries:
    response = execute_query(pyrel_query, context)
    responses_entity_id_format.append(response)

In [11]:
responses_nl_format = []

for response in responses_entity_id_format:

    if response != set():

        with graph.query() as select:
            lb = Labels()
            lb.lid.in_(response)
            label_names = select(lb, alias(lb.lname,"label"))
        responses_nl_format.append(set(label_names.results.get("label", [])))

    else:

        responses_nl_format.append(set())


In [12]:
for _, res in enumerate(responses_nl_format):
    
    res = ', '.join(res) if res != set() else "NULL"
    print(f"Answer of Question {_+1}: {res}")

Answer of Question 1: A Quiet Place
Answer of Question 2: Dunkirk
Answer of Question 3: The Royal Tenenbaums
Answer of Question 4: A Separation
Answer of Question 5: The Prestige, Dunkirk, Inception, Schindler's List, Saving Private Ryan, War Horse, The Dark Knight
Answer of Question 6: Men in Black, Jurassic World: Fallen Kingdom, Saving Private Ryan, Schindler's List, War Horse
Answer of Question 7: The Irishman, Righteous Kill, Heat
Answer of Question 8: Lee Smith
Answer of Question 9: Goodfellas, Jackie Brown
